In [92]:
import numpy as np

In [93]:
from sklearn.datasets import fetch_20newsgroups_vectorized
X, y = fetch_20newsgroups_vectorized(subset='train', 
                                     return_X_y=True, 
                                     remove=('headers', 'footers'))

In [94]:
X.shape, y.shape

((11314, 114751), (11314,))

# Préparation des données

Dans la suite, on va garder les 5 classes les plus représentées parmi les 1000 premiers exemples de `X`. Les exemples associés autres autres classes sont écartés. On obtient finalement un échantillon que vous appelez `X_g` de 306 exemples.  

In [95]:
nb_classes = 5
n_exemples = 1000

**Question**  Utiliser [np.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) avec l'argument `return_counts` pour identifier dans `labels` les classes que vous gardez.

In [96]:
X= X[:n_exemples]
y = y[:n_exemples]
# get most repeated classes
classes, counts = np.unique(y, return_counts=True)
classes = classes[np.argsort(counts)][::-1][:nb_classes]

In [97]:
classes

array([ 1, 13, 10,  6, 15])

In [98]:
#keep only the most repeated classes
mask = np.isin(y, classes)
X_g = X[mask]

y_g = y[mask]
X_g.shape

(306, 114751)

**Question** À l'aide de [np.in1d](https://numpy.org/doc/stable/reference/generated/numpy.in1d.htm), une ufunc qui teste l'appartenance à un ensemble, construire `X_g` et `y_g` les 306 exemples et leurs étiquettes.

In [99]:
# with np.in1d
mask = np.in1d(y, classes)
X_gbis = X[mask]
y_gbis = y[mask]
X_gbis.shape

(306, 114751)

Si vous n'avez pas réussi à faire cela, vous pouvez charger les données dans `pb.csv.xz`. (c'est un fichier compressé que panda sait lire).

# Encodage des classes

Dans la méthode de Zhou et al 2003, nous avons regardé l'identité entre un calcul itératif et une version analytique de ce calcul. Mais nous n'avons pas réellement défini une méthode de classification. En fait le problème de classification est résolu en prenant une fonction $F$ qui encode le problème de décision associé à la classification.

La méthode s'applique généralement aux problèmes multiclasses, avec un nombre $c$ de classes. 

Pour chaque noeud, la fonction $F$ associe un score à chaque classe possible dans $[0,\dots,c-1]$. Alors $F(i, k)$ est le score associé à la classe $k$ pour le noeud $i$. On fera une décision en prenant le score maximal associé à chaque noeud :

\begin{equation}
\mathop{\mathrm {argmax}}_{0\leq k< c} F(i,k)\hspace{5cm}(1)
\end{equation}

En fait $F$ est l'encodage de `y_g` par un `OneHotEncoder`.

**Question** Calculer `F_star` qui est l'encodage des classes à l'aide de cette méthode.

In [100]:
from sklearn.preprocessing import OneHotEncoder
y_g.reshape(-1,1)
enc = OneHotEncoder()
F_star = enc.fit_transform(y_g.reshape(-1,1)).toarray()
F_star.shape

(306, 5)

Dans ce problème semi-supervisé, la fonction $F$ est initialisée en mettant $F(i, k)=1$ pour tous les noeuds dont on connait l'étiquette associée $k$ et 0 partout ailleurs.  

**Question** Définir `F`

In [101]:
nb_labeles =  300

In [102]:
F = F_star.copy()
F[nb_labeles:, :] = 0
F

array([[0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

# Label spreading

In [103]:
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.neighbors import kneighbors_graph

**Question** Définir deux matrices `M_rbf` et `M_knn` pour calculer deux noyaux de similarité à partir 
- d'un noyau gaussien (on prendra $\gamma=20$), et 
- d'un $k$ plus proches voisins (on prendra $k=3$). 

In [104]:
M_rbf = rbf_kernel(X_g, gamma=20)
M_knn = kneighbors_graph(X_g, n_neighbors=3).toarray()

**Question** Définir une fonction `getS` qui prend en argument une matrice noyau et calcule la matrice $S$ associée à la méthode de Zhou et al 2003. NB: la diagonale de la matrice noyau doit être nulle. 

In [105]:
def getS(M) : 
    M = M - np.diag(np.diag(M))
    D = np.diag(M.sum(axis=1))
    return D**(-1/2) @ M @ D**(-1/2)


**Question** écrire une une fonction `zhou_iter` qui prend en argument, 
- une matrice noyau `M`, 
- une matrice `F` qui encode les classes (avec une partie supervisée et une partie non supervisée comme fait précédemment),
- une liste de classes (comme `labels`) qui correspond à chaque colonne de `F`,
- une valeur de $\alpha$ (0.2 par défaut) et  
- `max_iter` correspondant nombre maximal d'itérations. 

La fonction retourne le vecteur des classes correspondant au maximum définit par l'équation (1) ci-dessus. 

In [106]:
def Zhou_iter(M , f , labels , alpha = 0.2 , max_iter = 100) : 
    S = getS(M)
    f_star = f.copy()
    for _ in range(max_iter):
        if np.allclose(f_star, f):
            break
        else : 
            f_star = alpha * S @ f_star + (1 - alpha) * labels
        
    return np.argmax(f_star , axis = 0 )

**Question** Écrire maintenant une fonction `zhou_analytique` qui fait ce calcul mais à l'aide de l'expression analytique. Les paramètres sont identiques (hormis `max_iter`). La valeur retournée est identique.

In [107]:
def zhou_analytique(M, F, labels, alpha=0.2):
    return labels[np.argmax(np.linalg.inv(np.eye(M.shape[0])- alpha*getS(M))@F, axis=1)]

**Question** Calculer l'erreur en classification avec vos implémentations de la méthode de Zhou.

# Avec Sklearn

**Question** Refaire la même chose avec sklearn : calculer l'erreur de classification avec la méthode de Zhou implantée dans sklearn. (NB: le résultat peut être légèrement différent).

In [108]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score


In [109]:
label_prop_model = LabelPropagation(kernel='rbf', gamma=20, max_iter=1000)
label_prop_model.fit(X_g[:nb_labeles], y_g[:nb_labeles])
y_pred = label_prop_model.predict(X_g[nb_labeles:])
print("accuracy score : ", accuracy_score(y_g[nb_labeles:], y_pred))

accuracy score :  0.3333333333333333
